# **Superstore Analysis with SQL**
With growing demands and cut-throat competitions in the market, a Superstore Giant is seeking to understand what works best for them. They would like to **understand which products, regions, categories and customer segments they should target or avoid** (Taken from Kaggle).

The dataset is **already cleaned** and is **suitable for analysis**.

In [8]:
import sqlite3
import pandas as pd

csv_file = 'Superstore.csv'
df = pd.read_csv(csv_file, encoding='ISO-8859-1')

# Connect to an SQLite database (in-memory for temporary usage)
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

# Load the DataFrame into a SQL table
table_name = 'superstore_table'
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%Y', errors='coerce')
df.to_sql('superstore_table', connection, if_exists='replace', index=False)

query = "SELECT * FROM superstore_table LIMIT 5"
result = pd.read_sql_query(query, connection)
print(result)

   Row ID        Order ID           Order Date   Ship Date       Ship Mode  \
0       1  CA-2016-152156  2016-11-08 00:00:00  11/11/2016    Second Class   
1       2  CA-2016-152156  2016-11-08 00:00:00  11/11/2016    Second Class   
2       3  CA-2016-138688  2016-06-12 00:00:00   6/16/2016    Second Class   
3       4  US-2015-108966  2015-10-11 00:00:00  10/18/2015  Standard Class   
4       5  US-2015-108966  2015-10-11 00:00:00  10/18/2015  Standard Class   

  Customer ID    Customer Name    Segment        Country             City  \
0    CG-12520      Claire Gute   Consumer  United States        Henderson   
1    CG-12520      Claire Gute   Consumer  United States        Henderson   
2    DV-13045  Darrin Van Huff  Corporate  United States      Los Angeles   
3    SO-20335   Sean O'Donnell   Consumer  United States  Fort Lauderdale   
4    SO-20335   Sean O'Donnell   Consumer  United States  Fort Lauderdale   

   ... Postal Code  Region       Product ID         Category Sub-Cat

In [9]:
query = """
SELECT strftime('%Y', "Order Date") AS year,
       SUM(sales) AS total_sales,
       SUM(profit) AS total_profit
FROM superstore_table
GROUP BY year
ORDER BY year
"""

result = pd.read_sql_query(query, connection)
print(result)


   year  total_sales  total_profit
0  2014  484247.4981    49543.9741
1  2015  470532.5090    61618.6037
2  2016  609205.5980    81795.1743
3  2017  733215.2552    93439.2696


From the above query, it can be seen that there has been a **steady increase in total profits over the years**, despite the total sales dropping slightly from 2014-2015. This indicates an improvement in profit margin from 2014 to 2015.

In [15]:
query = """
SELECT strftime('%Y', "Order Date") AS year,
       CASE
           WHEN strftime('%m', "Order Date") IN ('01', '02', '03') THEN 'Q1'
           WHEN strftime('%m', "Order Date") IN ('04', '05', '06') THEN 'Q2'
           WHEN strftime('%m', "Order Date") IN ('07', '08', '09') THEN 'Q3'
           ELSE 'Q4'
       END AS quarter,
       SUM(Sales) AS total_sales,
       SUM(Profit) AS total_profit
FROM superstore_table
GROUP BY year, quarter
ORDER BY year, quarter
"""

result = pd.read_sql_query(query, connection)
print(result)


    year quarter  total_sales  total_profit
0   2014      Q1   74447.7960     3811.2290
1   2014      Q2   86538.7596    11204.0692
2   2014      Q3  143633.2123    12804.7218
3   2014      Q4  179627.7302    21723.9541
4   2015      Q1   68851.7386     9264.9416
5   2015      Q2   89124.1870    12190.9224
6   2015      Q3  130259.5752    16853.6194
7   2015      Q4  182297.0082    23309.1203
8   2016      Q1   93237.1810    11441.3708
9   2016      Q2  136082.3010    16390.3394
10  2016      Q3  143787.3622    15823.6048
11  2016      Q4  236098.7538    38139.8593
12  2017      Q1  123144.8602    23506.2026
13  2017      Q2  133764.3720    15499.2085
14  2017      Q3  196251.9560    26985.1325
15  2017      Q4  280054.0670    27448.7260


The store achieved its **peak profitability during the fourth quarter** and its **lowest profits in the first quarter** across most years, with 2017 being the only exception to this pattern.

In [17]:
query = """
SELECT region, SUM(sales) AS total_sales, SUM(profit) AS total_profits
FROM superstore_table
GROUP BY region
ORDER BY total_profits DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

    Region  total_sales  total_profits
0     West  725457.8245    108418.4489
1     East  678781.2400     91522.7800
2    South  391721.9050     46749.4303
3  Central  501239.8908     39706.3625


The **West region generated the highest total sales and profits**, whereas the **Central region recorded the lowest figures**.

In [20]:
query = """
SELECT region, ROUND((SUM(Profit)/ SUM(Sales)) * 100, 2) AS profit_margin
FROM superstore_table
GROUP BY region
ORDER BY profit_margin DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

    Region  profit_margin
0     West          14.94
1     East          13.48
2    South          11.93
3  Central           7.92


The **West has the highest profit margin** while the **Central has the lowest**.

In [22]:
query = """
SELECT State, SUM(Sales) as Total_Sales, SUM(Profit) as Total_Profits, ROUND((SUM(profit) / SUM(sales)) * 100, 2) as profit_margin
FROM superstore_table
GROUP BY State
ORDER BY Total_Profits DESC
LIMIT 10;
"""

result = pd.read_sql_query(query, connection)
print(result)

        State  Total_Sales  Total_Profits  profit_margin
0  California  457687.6315     76381.3871          16.69
1    New York  310876.2710     74038.5486          23.82
2  Washington  138641.2700     33402.6517          24.09
3    Michigan   76269.6140     24463.1876          32.07
4    Virginia   70636.7200     18597.9504          26.33
5     Indiana   53555.3600     18382.9363          34.33
6     Georgia   49095.8400     16250.0433          33.10
7    Kentucky   36591.7500     11199.6966          30.61
8   Minnesota   29863.1500     10823.1874          36.24
9    Delaware   27451.0690      9977.3748          36.35


**California, New York and Washington are our most profitable markets**, but their profit margins are lower than average, indicating that their sales are so high that their overall profits are still higher than states with higher profit margins.

In [23]:
query = """
SELECT State, SUM(Sales) as Total_Sales, SUM(Profit) as Total_Profits, ROUND((SUM(profit) / SUM(sales)) * 100, 2) as profit_margin
FROM superstore_table
GROUP BY State
ORDER BY Total_Profits ASC
LIMIT 10;
"""

result = pd.read_sql_query(query, connection)
print(result)

            State  Total_Sales  Total_Profits  profit_margin
0           Texas  170188.0458    -25729.3563         -15.12
1            Ohio   78258.1360    -16971.3766         -21.69
2    Pennsylvania  116511.9140    -15559.9603         -13.35
3        Illinois   80166.1010    -12607.8870         -15.73
4  North Carolina   55603.1640     -7490.9122         -13.47
5        Colorado   32108.1180     -6527.8579         -20.33
6       Tennessee   30661.8730     -5341.6936         -17.42
7         Arizona   35282.0010     -3427.9246          -9.72
8         Florida   89473.7080     -3399.3017          -3.80
9          Oregon   17431.1500     -1190.4705          -6.83


Among our markets, **Texas, Ohio, and Pennsylvania show the lowest profitability**. Texas and Pennsylvania are particularly concerning, as they generate substantial sales exceeding $100,000, with Texas even surpassing Washington's sales volume despite its poor profit performance.

In [24]:
query = """
SELECT Discount, AVG(Sales) AS Avg_Sales
FROM superstore_table
GROUP BY Discount
ORDER BY Discount;
"""

result = pd.read_sql_query(query, connection)
print(result)

    Discount   Avg_Sales
0       0.00  226.742074
1       0.10  578.397351
2       0.15  529.971567
3       0.20  209.076940
4       0.30  454.742974
5       0.32  536.794770
6       0.40  565.134874
7       0.45  498.634000
8       0.50  892.705152
9       0.60   48.150000
10      0.70   97.177708
11      0.80   56.545853


There **does not seem to a relationship between the discounts given and the average sales.**

In [25]:
query = """
SELECT category, SUM(discount) AS total_discount
FROM superstore_table
GROUP BY category
ORDER BY total_discount DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

          Category  total_discount
0  Office Supplies          947.80
1        Furniture          368.89
2       Technology          244.40


**Office supplies are the most discounted items followed by Furniture and Technology**

In [30]:
query = """
SELECT category, "Sub-Category", SUM(discount) AS total_discount
FROM superstore_table
GROUP BY category, "Sub-Category"
ORDER BY total_discount DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

           Category Sub-Category  total_discount
0   Office Supplies      Binders          567.00
1        Technology       Phones          137.40
2         Furniture  Furnishings          132.40
3         Furniture       Chairs          105.00
4   Office Supplies        Paper          102.60
5         Furniture       Tables           83.35
6   Office Supplies   Appliances           77.60
7   Office Supplies      Storage           63.20
8        Technology  Accessories           60.80
9   Office Supplies          Art           59.60
10        Furniture    Bookcases           48.14
11       Technology     Machines           35.20
12  Office Supplies       Labels           25.00
13  Office Supplies    Envelopes           20.40
14  Office Supplies    Fasteners           17.80
15  Office Supplies     Supplies           14.60
16       Technology      Copiers           11.00


**Binders receive significantly higher discounts compared to other products, with Phones and Furnishings being the next most discounted items, though at notably lower rates.**

In [31]:
query = """
SELECT category, SUM(sales) AS total_sales, SUM(profit) AS total_profit, ROUND(SUM(profit)/SUM(sales)*100, 2) AS profit_margin
FROM superstore_table
GROUP BY category
ORDER BY total_profit DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

          Category  total_sales  total_profit  profit_margin
0       Technology  836154.0330   145454.9481          17.40
1  Office Supplies  719047.0320   122490.8008          17.04
2        Furniture  741999.7953    18451.2728           2.49


**Technology and Office Supplies are the best in terms of profits**

In [32]:
query = """
SELECT region, category, SUM(sales) AS total_sales, SUM(profit) AS total_profit
FROM superstore_table
GROUP BY region, category
ORDER BY total_profit DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

     Region         Category  total_sales  total_profit
0      West  Office Supplies  220853.2490    52609.8490
1      East       Technology  264973.9810    47462.0351
2      West       Technology  251991.8320    44303.6496
3      East  Office Supplies  205516.0550    41014.5791
4   Central       Technology  170416.3120    33697.4320
5     South       Technology  148771.9080    19991.8314
6     South  Office Supplies  125651.3130    19986.3928
7      West        Furniture  252612.7435    11504.9503
8   Central  Office Supplies  167026.4150     8879.9799
9     South        Furniture  117298.6840     6771.2061
10     East        Furniture  208291.2040     3046.1658
11  Central        Furniture  163797.1638    -2871.0494


Of our most profitable category-region combinations, the **West region stands out with both Office Supplies and Technology** making the top three, **alongside Technology in the East**. However, while most categories generate positive profits across regions, the **Central region's Furniture sales operate at a loss**, making it our only unprofitable category-region combination.

In [33]:
query = """
SELECT state, category, SUM(sales) AS total_sales, SUM(profit) AS total_profit
FROM superstore_table
GROUP BY state, category
ORDER BY total_profit DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

          State         Category  total_sales  total_profit
0      New York       Technology  127483.5000    42186.7856
1    California  Office Supplies  142351.9480    37748.3847
2    California       Technology  159271.0820    29470.0368
3      New York  Office Supplies   90020.0420    25994.0829
4    Washington       Technology   50536.7100    15019.3435
..          ...              ...          ...           ...
137    Illinois  Office Supplies   19907.9060    -8354.1568
138    Illinois        Furniture   28274.5220    -9076.2894
139       Texas        Furniture   60593.2918   -10436.1419
140        Ohio       Technology   35675.9920   -12649.9401
141       Texas  Office Supplies   44490.5300   -18584.6434

[142 rows x 4 columns]


The table above shows the most and least performing categories in each of our states. **Technology in New York and Washington and Office Supplies in California are profitable overall**, while **Office supplies in Texas, Technology in Ohio and Furniture in Texas and Illinois are our biggest losses**.

In [36]:
query = """
SELECT "sub-category", SUM(sales) AS total_sales, SUM(profit) AS total_profit, ROUND(SUM(profit)/SUM(sales)*100, 2) AS profit_margin
FROM superstore_table
GROUP BY "sub-category"
ORDER BY total_profit DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

   Sub-Category  total_sales  total_profit  profit_margin
0       Copiers  149528.0300    55617.8249          37.20
1        Phones  330007.0540    44515.7306          13.49
2   Accessories  167380.3180    41936.6357          25.05
3         Paper   78479.2060    34053.5693          43.39
4       Binders  203412.7330    30221.7633          14.86
5        Chairs  328449.1030    26590.1663           8.10
6       Storage  223843.6080    21278.8264           9.51
7    Appliances  107532.1610    18138.0054          16.87
8   Furnishings   91705.1640    13059.1436          14.24
9     Envelopes   16476.4020     6964.1767          42.27
10          Art   27118.7920     6527.7870          24.07
11       Labels   12486.3120     5546.2540          44.42
12     Machines  189238.6310     3384.7569           1.79
13    Fasteners    3024.2800      949.5182          31.40
14     Supplies   46673.5380    -1189.0995          -2.55
15    Bookcases  114879.9963    -3472.5560          -3.02
16       Table

From our 17 subcategories nationwide, **Copiers, Phones, Accessories, and Paper generate our highest profits**, with Copiers and Paper showing particularly promising profit margins for future growth. On the other hand, **Tables, Bookcases, and Supplies are operating at a loss**, failing to reach the break-even point.

In [39]:
query = """
SELECT state, "sub-category", SUM(sales) AS total_sales, SUM(profit) AS total_profit
FROM superstore_table
GROUP BY state, "sub-category"
ORDER BY total_profit DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

              State Sub-Category  total_sales  total_profit
0          New York     Machines    43183.500    17320.0762
1          New York       Phones    47502.620    13399.1892
2          New York      Binders    32456.912    11096.0260
3        California  Accessories    37255.010    11095.6565
4          Michigan      Binders    22821.970    11079.5050
..              ...          ...          ...           ...
651  North Carolina     Machines    12620.655    -5384.8086
652           Texas   Appliances     2407.814    -6147.2225
653        Illinois      Binders     4538.546    -7204.3242
654            Ohio     Machines     8978.238   -11770.9447
655           Texas      Binders     9042.676   -14705.0738

[656 rows x 4 columns]


**New York sees strong performance in Machines, Phones, and Binders**. **California excels in Accessories and Binders**, while **Michigan favors Binders. Texas and Illinois experience significant losses in Binders**, and **Machines are entirely unprofitable in Ohio**.

In [42]:
query = """
SELECT "Product Name", SUM(sales) AS total_sales, SUM(profit) AS total_profit
FROM superstore_table
GROUP BY "Product Name"
ORDER BY total_profit DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

                                           Product Name  total_sales  \
0                 Canon imageCLASS 2200 Advanced Copier    61599.824   
1     Fellowes PB500 Electric Punch Plastic Comb Bin...    27453.384   
2                  Hewlett Packard LaserJet 3310 Copier    18839.686   
3                    Canon PC1060 Personal Laser Copier    11619.834   
4     HP Designjet T520 Inkjet Large Format Printer ...    18374.895   
...                                                 ...          ...   
1845  Bush Advantage Collection Racetrack Conference...     9544.725   
1846  Chromcraft Bull-Nose Wood Oval Conference Tabl...     9917.640   
1847          Cubify CubeX 3D Printer Triple Head Print     7999.980   
1848          Lexmark MX611dhe Monochrome Laser Printer    16829.901   
1849          Cubify CubeX 3D Printer Double Head Print    11099.963   

      total_profit  
0       25199.9280  
1        7753.0390  
2        6983.8836  
3        4570.9347  
4        4094.9766  
...      

Profitability heavily relies on copiers, machines, and printers. The **Canon imageClass 2200 Advanced Copier, Fellowes PB500 Electric Punch, and Hewlett Packard LaserJet 3310 Copier are the top-performing models**, contributing significantly to our bottom line.

In [43]:
query = """
SELECT "Product Name", SUM(sales) AS total_sales, SUM(profit) AS total_profit
FROM superstore_table
GROUP BY "Product Name"
ORDER BY total_profit ASC;
"""

result = pd.read_sql_query(query, connection)
print(result)

                                           Product Name  total_sales  \
0             Cubify CubeX 3D Printer Double Head Print    11099.963   
1             Lexmark MX611dhe Monochrome Laser Printer    16829.901   
2             Cubify CubeX 3D Printer Triple Head Print     7999.980   
3     Chromcraft Bull-Nose Wood Oval Conference Tabl...     9917.640   
4     Bush Advantage Collection Racetrack Conference...     9544.725   
...                                                 ...          ...   
1845  HP Designjet T520 Inkjet Large Format Printer ...    18374.895   
1846                 Canon PC1060 Personal Laser Copier    11619.834   
1847               Hewlett Packard LaserJet 3310 Copier    18839.686   
1848  Fellowes PB500 Electric Punch Plastic Comb Bin...    27453.384   
1849              Canon imageCLASS 2200 Advanced Copier    61599.824   

      total_profit  
0       -8879.9704  
1       -4589.9730  
2       -3839.9904  
3       -2876.1156  
4       -1934.3976  
...      

The **Cubify CubeX 3D Printer Double Head Print, Lexmark MX611dhe Monochrome Laser Printer, and Cubify CubeX 3D Printer Triple Head Print are our most unprofitable products**.

In [44]:
query = """
SELECT segment, SUM(sales) AS total_sales, SUM(profit) AS total_profit
FROM superstore_table
GROUP BY segment
ORDER BY total_profit DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

       Segment   total_sales  total_profit
0     Consumer  1.161401e+06   134119.2092
1    Corporate  7.061464e+05    91979.1340
2  Home Office  4.296531e+05    60298.6785


The **consumer segment brings in the most profit followed by Corporate and then Home office**.

In [47]:
query = """
SELECT state, COUNT(DISTINCT "Customer ID") AS total_customers
FROM superstore_table
GROUP BY state
ORDER BY total_customers DESC;
"""

result = pd.read_sql_query(query, connection)
print(result)

                   State  total_customers
0             California              577
1               New York              415
2                  Texas              370
3           Pennsylvania              257
4               Illinois              237
5             Washington              224
6                   Ohio              202
7                Florida              181
8         North Carolina              122
9               Virginia              107
10              Michigan              106
11               Arizona              100
12             Tennessee               84
13               Georgia               83
14              Colorado               75
15               Indiana               70
16         Massachusetts               62
17            New Jersey               61
18              Kentucky               58
19             Wisconsin               52
20                Oregon               51
21              Maryland               45
22              Delaware          

**California, New York, and Texas boast the highest customer concentration.** Conversely, **Wyoming, North Dakota, and West Virginia exhibit the lowest customer activity**.

In [48]:
query = """
SELECT "Customer ID",
SUM(sales) AS total_sales,
SUM(profit) AS total_profit
FROM superstore_table
GROUP BY "Customer ID"
ORDER BY total_sales DESC
LIMIT 15;
"""

result = pd.read_sql_query(query, connection)
print(result)

   Customer ID  total_sales  total_profit
0     SM-20320    25043.050    -1980.7393
1     TC-20980    19052.218     8981.3239
2     RB-19360    15117.339     6976.0959
3     TA-21385    14595.620     4703.7883
4     AB-10105    14473.571     5444.8055
5     KL-16645    14175.229      806.8550
6     SC-20095    14142.334     5757.4119
7     HL-15040    12873.298     5622.4292
8     SE-20110    12209.438     2650.6769
9     CC-12370    12129.072     2177.0493
10    TS-21370    11891.751     2371.7144
11    GT-14710    11820.120     2163.4269
12    BM-11140    11789.630    -1659.9581
13    SV-20365    11470.950     1199.4242
14    CJ-12010    11164.974      858.7414


**Customer ID 'SM-20320'** presents an interesting case. This customer has **spent the most** with us, yet they are **not profitable**. This situation warrants further investigation to understand the underlying reasons behind this discrepancy.

# **Key Findings & Recommendations**

## **Overall Performance:**

**Positive Trajectory:** Profits have shown consistent growth, with a minor setback in 2015. This upward trend should be maintained.

**Seasonal Strength:** Q4 consistently delivers the highest profits. Maximize this by ensuring adequate inventory, intensifying marketing efforts, and enhancing customer service during the October-December festive season.

## **Regional Performance:**

**West Dominates:** The West region is the most profitable, followed by East, South, and Central.

**Central Region Focus:** While the Central region generates significant sales (>$100,000 more than South), profitability lags. Analyze and address the root causes to improve performance. Consider reallocating some resources from Central to the more profitable West region.

## **Market-Level Performance:**

**Key Markets:** California, New York, and Washington are high-profit, high-sales markets. Focus on strengthening presence in these key states.

**Underperforming Markets:** Texas, Ohio, and Pennsylvania exhibit low profitability. Evaluate the need for reduced presence or potential market exit, especially in Texas and Pennsylvania where high sales fail to translate into profits.

**City-Level Focus:** Prioritize high-profit cities like New York City, Los Angeles, and Seattle. Dominating key cities can significantly influence state-level performance.

**Texas Re-evaluation:** The significant losses in Texas, particularly in two of the top three loss-making cities, necessitate a critical review of the company's strategy in this state.

## **Product Category Performance:**

**Top Performers:** Technology and Office Supplies demonstrate high profitability and attractive profit margins.

**Furniture Review:** While generating sales, Furniture exhibits low profitability and margins. Conduct a thorough analysis to identify areas for improvement within this category.

## **Regional Optimization:**

**West:** Increase the focus on Office Supplies and Technology.

**East:** Increase the focus on Office Supplies and Technology.

**Central:** Minimize the focus on Furniture.

## **State-Level Adjustments:**

**Increase Focus:** Enhance the availability of Technology and Office Supplies in New York and California.

**Reduce Focus:** Significantly reduce the availability of Office Supplies in Texas, Technology in Ohio, and Furniture in Texas and Illinois.

## **Subcategory Performance:**

**High-Profit Subcategories:** Copiers, Phones, Accessories, and Paper demonstrate strong profitability and long-term potential. Prioritize these products.

**Loss-Making Subcategories:** Tables, Bookcases, and Supplies exhibit significant losses. Minimize investment in these areas, particularly Tables, which incur the highest losses.

## **Regional Subcategory Focus:**

**Increase Focus:** Prioritize Copiers, Accessories, and Binders in the West and East.

**Reduce Focus:** Minimize focus on Tables in all regions and Furnishings in the Central region.

## **State-Level Subcategory Focus:**

**Increase Focus:** Emphasize Machines, Phones, and Binders in New York; Accessories and Binders in California; and Binders in Michigan.

**Reduce Focus:** Decrease stock of Binders in Texas and Illinois, and Machines in Ohio.

## **Product-Level Performance:**

**Top-Performing Products:** Canon imageClass 2200 Advanced Copier, Fellowes PB500 Electric Punch, and Hewlett Packard LaserJet 3310 Copier consistently deliver high profits. Maintain adequate inventory levels.

**Loss-Making Products:** Cubify CubeX 3D Printer Double Head Print, Lexmark MX611dhe Monochrome Laser Printer, and Cubify CubeX 3D Printer Triple Head Print consistently operate at a loss. Consider discontinuing these products.

## **Customer Segment Performance:**

**Consumer Focus:** The Consumer segment is the most profitable. Prioritize this segment while maintaining focus on Corporate and Home Office segments.

## **Customer Base Analysis:**

**Key Customer Markets:** California, New York, and Texas have the highest customer concentration.

**Texas Re-evaluation:** Despite having a significant customer base, Texas exhibits low profitability. Conduct a thorough analysis to determine the most effective strategy for this market.

**Focus on Key States:** Prioritize customer acquisition and retention strategies in California and New York.

## **Overall Recommendations:**

- Conduct a thorough market analysis in Texas to determine the optimal course of action.
- Focus on improving profitability in the Central region.
- Prioritize high-growth areas like California, New York, and Washington.
- Optimize product mix across all regions and states.
- Continuously monitor and adjust product offerings based on performance data.
- Invest in customer service and marketing efforts, particularly during peak seasons.